In [1]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy.linalg import cholesky
# from math import e as euler

First get weather data and metadata for a desired location (latitude and logitude)

In [2]:
# change to desired values (currently Miami)
latitude = 25.783388
longitude = -80.189029

API_KEY = credentials.API_KEY # my personal NREL api key
email ='tobin.ford@nrel.gov' # replace these values with your appropriate information and remove or comment out first line of first block (import credentials)

# reads NSRDB data 
weather_df, meta = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, email, names='2019', map_variables=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pvlib/iotools/psm3.py:183: pvlibDeprecationWarning: The ``get_psm3`` function will default to leap_day=True starting in pvlib 0.11.0. Specify leap_day=True to enable this behavior now, or specify leap_day=False to hide this warning.
  warnings.warn(


User has 3 parameters for initial implementation: See Kempe's "Deg Miami" tab in excel<br>



activation energy, Ea <br>
irradiance relation, x<br>
ln(R0)<br>

|           |   Ea   |   x   | ln(R0) |
|:---------:|:-----:|:----:|:-------:|
|   **Ea**  |   1   |   a  |   b     |
|   **x**   |   a   |   1  |   c     |
| **ln(R0)**|   b   |   c  |   1     |

Notice symmetry across diagonal <br>

In [3]:
# USER ENTERED VALUES
# Correlation Coefficients
Ea_X = 0.0269
Ea_lnR0 = -0.9995
X_lnR0 = -0.0400

# Activation Energy
mean_Ea = 129 # average
sd_Ea = 3.4 # standard deviation

# Irradiance relation
mean_X = 0.0341 # average
sd_X = 0.0992757 # standard deviation

# ln(R0)
mean_R0 = 13.7223084 
sd_R0 = 2.47334772

# number of iterations
n = 20000


In [4]:
# notice symmetry of matrix
A = np.array([[1, Ea_X, Ea_lnR0],
              [Ea_X, 1, X_lnR0],
              [Ea_lnR0, X_lnR0, 1]])

# conceptually similar to the square root of a matrix
A_decomp = cholesky(A, lower=True)

# generating standard distribution of points for each
Ea, X, lnR0 = np.random.normal(mean_Ea, sd_Ea, 20_000), np.random.normal(mean_X, sd_X, 20_000), np.random.normal(mean_R0, sd_R0, 20_000)

# transpose built in so I don't have to do it with another function
samples_matrix = np.array([Ea, X, lnR0])



In [5]:
# correlated stats pre-input to function

# not entirely sure what I should do here,
# do i multiply the transpose by the lower cholesky to get correlated input data for the equation on the spreadsheet
# using MonteCarloEaLnRoX

# this gives us correlated input samples
# looks like what kempe's code was doing in "MonteCarloEaLnRoX" but it is hard to tell what's happening in his code
correlated_samples = np.matmul(A_decomp, samples_matrix)

correlated_df = pd.DataFrame(correlated_samples.T, columns=['Ea', 'X', 'lnR0'])

# is back tracking on by default
sol_pos = pvdeg.spectral.solar_position(weather_df, meta)
poa_irradiance = pvdeg.spectral.poa_irradiance(weather_df, meta)
temp_mod = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_irradiance, conf='open_rack_glass_polymer')
# we only care about global irradiance in this case


In [6]:
correlated_df

,Ea,X,lnR0
0,133.167750,3.622784,-132.611005
1,129.867005,3.559062,-129.267965
2,125.059050,3.412700,-124.708310
3,131.150349,3.575740,-130.610875
4,127.401000,3.371354,-126.980172
...,...,...,...
19995,131.917940,3.543365,-131.408835
19996,134.385363,3.600968,-133.887352
19997,122.359233,3.322717,-121.927659
19998,128.641625,3.588067,-128.204425


In [15]:
def mikeArrhenius(weather_df, samples):
# Ea1 = Ea(Z) / 8.31446261815324E-03 'This reduces the amount of calculations in the loop
# Degradation(Z) = Degradation(Z) + Exp(LnRo(Z)) * Exp(-Ea1 / (273.15 + T(Y, 1))) * ((Irr(Y, 1) / 1000) ^ X(Z))

    # FUNCTION for x,y,z in zip(correlated_df['X'], correlated_df['Ea'], corrleated_df['R0'])
    d = np.exp(samples['lnR0']) * np.exp((samples['Ea'] / 8.31446261815324E-03) / (273.15 + weather_df['temp_air'])) * (weather_df['ghi'] / 1000 ** samples['X'])
    
    return pd.DataFrame(d)

In [23]:
def mikeArrhenius2(weather_df, samples): # not using argument properly, it didnt like when i did this

    #['Ea', 'X', 'R0'] where Ea = 'x', X = 'y', and lnR0 = 'z'
    # why does kempe have a negative in front of activation energy
    # it makes my values even smaller 10^-80 instead of 10^-40
    # should be cell temp not temp_air as gathered from macros sheet, *see notes*

    # modify to be cell_temp instead of air_temp and check input values, seens like a likely culprit
    d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + weather_df['temp_air'])) * (weather_df['ghi'] / 1000 ** y)).mean() for x, y, z in zip(samples['Ea'], samples['X'], samples['lnR0'])] 
    # added .mean(), is this right

    return pd.DataFrame(d)

In [24]:
#temp1 = mikeArrhenius(weather_df=weather_df, samples=correlated_df.iloc[0]) # works mostly as desired

#temp2 = mikeArrhenius2(weather_df=weather_df, samples=correlated_df.iloc[0]) # gets mad if i pass as value
temp2 = mikeArrhenius2(weather_df=weather_df, samples=correlated_df) # gets mad if i pass as value

# difference is probably something about how I am passing values to the functions,
# the original function takes data as arguments but the second just pulls them from globalscope, this could cause undesired behavior

In [25]:
temp2.head(20)
# what are the units on this 
# these are significantly different than the calculated rates on deg miami tab n*10^-a
# ex 5.2*10^-6
# where could the error be coming from 

,0
0,1.226044e-43
1,1.425384e-42
2,5.396606e-41
3,5.562113e-43
4,1.901878e-41
5,3.305284e-43
6,3.736149e-43
7,6.851718e-41
8,1.087712e-41
9,2.420781e-43
